In [1]:
# Creating Conda envrionment:  conda create -n "env_DataLit" python=3.9 plotly pandas numpy nbformat
# Or: 
# Activating conda env: conda activate env_DataLit


In [ ]:
import plotly.graph_objects as go
import pandas as pd 
import numpy as np 
import plotly.express as px

from ..util.ECA import load_dataset, get_violin

In [4]:

def get_violin(df, interval):
    fig = go.Figure()

    periods = sorted(df[interval].unique())
    for period in periods:
        fig.add_trace(go.Violin(y=df[df[interval] == period]['SPEED_MA'], name=f'{period}', box_visible=False, meanline_visible=True))
        
    fig.update_layout(
        title='Violin Plots of Weekly Moving Average of Wind Speed for Each {} Over the Years 1959-2023'.format(interval),
        xaxis_title='{}'.format(interval),
        yaxis_title='MA Wind Speed'
    )
    
    fig.update_layout(yaxis_range=[0, 220])

    return fig

def load_dataset(path, ma_window=7, max_window=30, is_random=False):
    df = pd.DataFrame()
    df = pd.DataFrame(columns=['SPEED', 'YEAR', 'MONTH', 'DAY', 'DECADE'])
    # df = pd.DataFrame(columns=['STAID', 'SOUID','DATE','FG','Q_FG'])
    star_parsing = False 
    with open(path, "r") as file:
        ith_row = 0
        while True:
            line = file.readline() 
            
            if len(line) == 0:
                break

            elif line.startswith(' STAID, SOUID,    DATE,   FG, Q_FG'):
                star_parsing = True
            elif star_parsing:
                fields = line.rstrip().strip().split(',')

                STAID = fields[0]
                SOUID = fields[1]
                DATE = fields[2]
                FG = int(fields[3])
                Q_FG = fields[4]
                YEAR = int(DATE[0:4])
                MONTH = int(DATE[4:6])
                DAY = int(DATE[6:8])
                DECADE = int(str(YEAR)[0:-1] + '0')

                if int(Q_FG) != 9:
                    # df.loc[ith_row] = [STAID, SOUID, DATE, FG, Q_FG]
                    df.loc[ith_row] = [FG, YEAR, MONTH, DAY, DECADE]
                    ith_row += 1

    if is_random:
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    df['SPEED_MA'] = df['SPEED'].rolling(ma_window).mean()
    df['SPEED_MAX'] = df['SPEED'].rolling(max_window).max() # not the prettiest way but is it correct?!
    df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])
    df['DAY_OF_YEAR'] = df['DATE'].dt.dayofyear


    return df

In [5]:
# Loading Data
path =  'C:/Users/gwend/Downloads/ECA_non-blended_custom/FG_SOUID115639.txt'

df = load_dataset(path, is_random=False) # Check it with True :'(

In [20]:
# Plotting Violin to invistigate distributions

fig_month = get_violin(df, 'MONTH')
fig_year = get_violin(df, 'YEAR')
fig_decade = get_violin(df, 'DECADE')

fig_month.show()
fig_year.show()
fig_decade.show()


In [4]:
# Scatter Plot of MA wind speed color sheme is by YEAR

fig = px.scatter(df, x='DAY_OF_YEAR', y='SPEED_MA', color='DECADE', title='Weekly Moving Average of Wind Speed Over Years by Day of the Year',color_continuous_scale=px.colors.sequential.Inferno)


fig.update_xaxes(title='Day of the Year')
fig.update_yaxes(title='Wind Speed')
fig.update_traces(opacity=.4)
fig.update_traces(marker={'size': 6})
fig.update_layout(yaxis_range=[0, 220])
fig.show()

In [6]:
# Scatter Plot of MA wind speed color sheme is by YEAR

fig = px.scatter(df, x='DAY_OF_YEAR', y='SPEED_MA', color='YEAR', title='Weekly Moving Average of Wind Speed Over Years by Day of the Year',color_continuous_scale=px.colors.sequential.Inferno)


fig.update_xaxes(title='Day of the Year')
fig.update_yaxes(title='Wind Speed')
fig.update_traces(opacity=.4)
fig.update_traces(marker={'size': 6})
fig.update_layout(yaxis_range=[0, 220])
fig.show()

In [7]:
# MAximum wind speed over Period

wind_data = df

fig = px.scatter(df, x='MONTH', y='SPEED_MAX', color='YEAR', title='Monthly Maximum Wind Speed Over Years by Day of the Year',color_continuous_scale=px.colors.sequential.Inferno)

fig.update_xaxes(title='Month')
fig.update_yaxes(title='Monthly Maximum Wind Speed')
fig.update_traces(opacity=.1, marker={'size': 10})
fig.update_layout(yaxis_range=[0, 220])
fig.show()